In [47]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.metrics import accuracy_score

import xgboost as xgb

import lightgbm as lgb
#os.chdir('C:/Users/BTHANISH/Documents/Thanish/Competition/Machine hack/Predicting Food Delivery Time')
os.chdir('E:/Thanish/Data science/Machine Hack/Predicting Food Delivery Time')

In [48]:
DF_train = pd.read_excel('Data_Train.xlsx')
DF_test= pd.read_excel('Data_Test.xlsx')

#Combining train and test
DF_test['Delivery_Time'] = None
DF_prod = pd.concat([DF_train, DF_test]).reset_index(drop = True)

print(DF_train.shape, DF_test.shape)

(11094, 9) (2774, 9)


In [49]:
DF_prod.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


In [50]:
#Dropping few rows
print(DF_prod.shape)
DF_prod = DF_prod.loc[DF_prod['Average_Cost'] != 'for',:]
DF_prod = DF_prod.loc[DF_prod['Rating'] != 'Temporarily Closed',:]
print(DF_prod.shape)

(13868, 9)
(13865, 9)


In [51]:
# Replacing the rupees 
DF_prod['Average_Cost'] = DF_prod.Average_Cost.str.replace('₹|,', '').astype('int')
DF_prod['Minimum_Order'] = DF_prod.Minimum_Order.str.replace('₹|,', '').astype('int')
print(DF_prod.dtypes)

Restaurant       object
Location         object
Cuisines         object
Average_Cost      int32
Minimum_Order     int32
Rating           object
Votes            object
Reviews          object
Delivery_Time    object
dtype: object


In [53]:
DF_prod['restaurant_type'] = np.where(DF_prod.Rating == '-', 3, 
                                     np.where(DF_prod.Rating == 'Opening Soon', 2, 
                                             np.where(DF_prod.Rating == 'NEW', 1, 0)))

DF_prod.Rating[DF_prod.Rating.isin(['-', 'Opening Soon', 'NEW'])] = -999
DF_prod.Rating = DF_prod.Rating.astype('float')

DF_prod.Votes[DF_prod.Votes.isin(['-'])] = -999
DF_prod.Votes = DF_prod.Votes.astype('float')

DF_prod.Reviews[DF_prod.Reviews.isin(['-'])] = -999
DF_prod.Reviews = DF_prod.Reviews.astype('float')


C:\Users\Bellagio\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Bellagio\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Bellagio\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [54]:
# Resetting the index
DF_prod = DF_prod.reset_index(drop = True)
DF_prod.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,restaurant_type
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",200,50,3.5,12.0,4.0,30 minutes,0
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",100,50,3.5,11.0,4.0,30 minutes,0
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",150,50,3.6,99.0,30.0,65 minutes,0
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",250,99,3.7,176.0,95.0,30 minutes,0
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",200,99,3.2,521.0,235.0,65 minutes,0


# Feature Engineering

In [55]:
# votes, reviews, rating multiple
DF_prod['votes_review_rating'] = DF_prod.Votes*DF_prod.Reviews*DF_prod.Rating


In [56]:
DF_prod['Cusine_len'] = DF_prod.Cuisines.apply(lambda x : len(x.replace(" ", " ").split(",")))

DF_prod['city'] = DF_prod['Location'].apply(lambda x : np.char.strip(x.split(','))[-1])
actual_city = {'Noida' : 'Noida', 
               'Gurgaon' : 'Gurgoan',
               'Gurgoan' : 'Gurgoan',
               'Mumbai CST Area' : 'Mumbai',
               'Mumbai Central' : 'Mumbai',
               'Mumbai' : 'Mumbai',
               'Pune' : 'Pune',
               'Maharashtra' : 'Pune',
               'Pune University' : 'Pune',
               'Timarpur' : 'Delhi',
               'Delhi' : 'Delhi',
               'Delhi Cantt.' : 'Delhi',
               'Delhi University-GTB Nagar' : 'Delhi',
               'India Gate' : 'Delhi',
               'Whitefield' : 'Banglore', 
               'Marathalli' : 'Banglore',
               'Majestic' : 'Banglore',
               'Bangalore' : 'Banglore',
               'Electronic City' : 'Banglore',
               'Hyderabad' : 'Hyderabad',
               'Begumpet' : 'Hyderabad',
               'Kolkata' : 'Kolkata'
               }

DF_prod['new_city'] = DF_prod[['city']].applymap(actual_city.get)
DF_prod.drop(['city'], axis = 1, inplace = True)

DF_prod.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,restaurant_type,votes_review_rating,Cusine_len,new_city
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",200,50,3.5,12.0,4.0,30 minutes,0,168.0,5,Pune
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",100,50,3.5,11.0,4.0,30 minutes,0,154.0,2,Banglore
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",150,50,3.6,99.0,30.0,65 minutes,0,10692.0,3,Mumbai
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",250,99,3.7,176.0,95.0,30 minutes,0,61864.0,3,Noida
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",200,99,3.2,521.0,235.0,65 minutes,0,391792.0,2,Banglore


In [57]:
#Adding average restaurant data
Average_restaurant_rating = DF_prod.loc[DF_prod.Rating != -999 ,:].groupby('Restaurant')['Rating'].agg([('avg_restaurant_Rating', 'mean')
                                                                                                        ,('median_restaurant_Rating', 'median')
                                                                                                        ]).reset_index()
Average_restaurant_Review = DF_prod.loc[DF_prod.Rating != -999 ,:].groupby('Restaurant')['Reviews'].agg([('avg_restaurant_Reviews', 'mean')
                                                                                                         ,('median_restaurant_Reviews', 'median'),
                                                                                                         ]).reset_index()
Average_restaurant_Votes = DF_prod.loc[DF_prod.Rating != -999 ,:].groupby('Restaurant')['Votes'].agg([('avg_restaurant_Votes', 'mean')
                                                                                                      ,('median_restaurant_Votes', 'median')
                                                                                                      ]).reset_index()

DF_prod = pd.merge(DF_prod, Average_restaurant_rating, how = 'left', left_on = 'Restaurant', right_on = 'Restaurant')
DF_prod = pd.merge(DF_prod, Average_restaurant_Review, how = 'left', left_on = 'Restaurant', right_on = 'Restaurant')
DF_prod = pd.merge(DF_prod, Average_restaurant_Votes, how = 'left', left_on = 'Restaurant', right_on = 'Restaurant')

columns = ['avg_restaurant_Rating', 'avg_restaurant_Reviews', 'avg_restaurant_Votes'
           ,'median_restaurant_Rating', 'median_restaurant_Reviews', 'median_restaurant_Votes'
           ]

DF_prod[columns] = DF_prod[columns].fillna(-999)
           
DF_prod.head(10)

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,restaurant_type,votes_review_rating,Cusine_len,new_city,avg_restaurant_Rating,median_restaurant_Rating,avg_restaurant_Reviews,median_restaurant_Reviews,avg_restaurant_Votes,median_restaurant_Votes
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",200,50,3.5,12.0,4.0,30 minutes,0,168.0,5,Pune,3.5000,3.5,4.00,4.0,12.00,12.0
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",100,50,3.5,11.0,4.0,30 minutes,0,154.0,2,Banglore,3.1250,3.0,9.25,11.0,19.25,22.0
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",150,50,3.6,99.0,30.0,65 minutes,0,10692.0,3,Mumbai,3.6000,3.6,30.00,30.0,99.00,99.0
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",250,99,3.7,176.0,95.0,30 minutes,0,61864.0,3,Noida,3.7000,3.7,95.00,95.0,176.00,176.0
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",200,99,3.2,521.0,235.0,65 minutes,0,391792.0,2,Banglore,3.2000,3.2,235.00,235.0,521.00,521.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,ID_3468,"D-Block, Sector 63, Noida",Fast Food,100,50,3.3,13.0,4.0,30 minutes,0,171.6,1,Noida,3.3000,3.3,4.00,4.0,13.00,13.0
96,ID_3797,"Delhi High Court, India Gate",Fast Food,200,50,3.3,25.0,16.0,30 minutes,0,1320.0,1,Delhi,3.3000,3.3,16.00,16.0,25.00,25.0
97,ID_6224,"D-Block, Sector 63, Noida",Chinese,100,50,3.5,24.0,1.0,30 minutes,0,84.0,1,Noida,3.5000,3.5,1.00,1.0,24.00,24.0
98,ID_1124,Delhi University-GTB Nagar,"Burger, Fast Food",100,50,-999.0,-999.0,-999.0,30 minutes,3,-997002999.0,2,Delhi,-999.0000,-999.0,-999.00,-999.0,-999.00,-999.0


In [58]:
# Total restaurant branches
DF_prod['total_branches'] = DF_prod.groupby(['Restaurant']).new_city.transform('count')

In [59]:
# Creating one hot encoding of the cuisines

available_cuisines = list(DF_prod.Cuisines.apply(lambda x : x.split(",")))
merged = list(itertools.chain.from_iterable(available_cuisines))
merged = np.sort(np.unique(np.char.lstrip(merged)))
print(merged)

cuisine_DF = pd.DataFrame(0, index=np.arange(len(DF_prod)), columns = merged)
print(cuisine_DF.shape)
cuisine_DF.head()

# Filling the Cusines OHE columns
for i in range(len(DF_prod)):    
    print(i)
    cuisine_list = DF_prod.Cuisines[i:(i+1)].apply(lambda x : np.char.strip(x.split(","))).tolist()
    cuisine_list = cuisine_list[0]
    
    for cuisine in cuisine_list:
        cuisine_DF.loc[i,cuisine] = 1
        
# cuisine_DF.to_csv('cuisine_DF.csv', index = False)

# cuisine_DF = pd.read_csv('cuisine_DF.csv')
cuisine_DF.rename(columns = {'Poké' : 'Poke'}, inplace = True)
cuisine_DF.tail()

,Afghan,African,American,Andhra,Arabian,Asian,Assamese,Awadhi,BBQ,Bakery,...,Street Food,Sushi,Tamil,Tea,Tex-Mex,Thai,Tibetan,Turkish,Vietnamese,Wraps
13860,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13862,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13863,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
13864,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [60]:
# Merging the cuisines OHE to the main dataframe

print(DF_prod.shape)
DF_prod = pd.merge(DF_prod, cuisine_DF, left_index=True, right_index=True)
print(DF_prod.shape)

(13865, 20)
(13865, 121)


In [61]:
# Separating the numerical and categorical columns

dep_column = 'Delivery_Time'

cat_columns = DF_prod.select_dtypes(include = ['object']).columns.difference([dep_column])
int_columns = DF_prod.select_dtypes(include = ['int32']).columns

print(cat_columns)

Index(['Cuisines', 'Location', 'Restaurant', 'new_city'], dtype='object')


In [62]:
# Convert cat columns to numerics
LE = LabelEncoder()
DF_prod[cat_columns] = DF_prod[cat_columns].apply(lambda x : LE.fit_transform(x))


In [63]:
# Separating the training and testing data

train_prod = DF_prod.loc[~DF_prod.Delivery_Time.isnull(), ]
test_prod  = DF_prod.loc[DF_prod.Delivery_Time.isnull(), ]

# Label encode the dependent column to reverse encode later
Dep_encoder = LabelEncoder()
Dep_encoder.fit(train_prod[dep_column])
train_prod[dep_column] = Dep_encoder.transform(train_prod[dep_column])

print(train_prod.shape, test_prod.shape)
train_prod.head()

(11091, 121) (2774, 121)


C:\Users\Bellagio\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,restaurant_type,...,Street Food,Sushi,Tamil,Tea,Tex-Mex,Thai,Tibetan,Turkish,Vietnamese,Wraps
0,5914,10,1085,200,50,3.5,12.0,4.0,3,0,...,0,0,0,0,0,0,0,0,0,1
1,2093,30,1215,100,50,3.5,11.0,4.0,3,0,...,0,0,0,0,0,0,0,0,0,0
2,663,19,1287,150,50,3.6,99.0,30.0,5,0,...,1,0,0,0,0,0,0,0,0,0
3,5478,28,1525,250,99,3.7,176.0,95.0,3,0,...,0,0,0,0,0,0,0,0,0,0
4,5694,26,423,200,99,3.2,521.0,235.0,5,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
# Splitting the data to local train and test
not_in_testset = ['Tex-Mex', 'Tamil', 'Sri Lankan', 'South American', 'Portuguese', 
                  'Nepalese', 'Konkan', 'Israeli', 'Indonesian', 'Greek', 'Gujarati', 
                  'Charcoal Chicken', 'Cantonese', 'Bubble Tea','Bohri']

indep = DF_prod.columns.difference([dep_column
                                    , 'total_branches'
                                    , 'Top_restaurant'
                                    ,'votes_review_rating'
                                   ] 
                                  )
print(indep)

np.random.seed(100)
train_local_X, test_local_X, train_local_Y, test_local_Y = train_test_split(train_prod[indep], 
                                                                            train_prod[dep_column],
                                                                            test_size = 0.3)

train_local_X.shape, train_local_Y.shape, test_local_X.shape, test_local_Y.shape

Index(['Afghan', 'African', 'American', 'Andhra', 'Arabian', 'Asian',
       'Assamese', 'Average_Cost', 'Awadhi', 'BBQ',
       ...
       'Votes', 'Wraps', 'avg_restaurant_Rating', 'avg_restaurant_Reviews',
       'avg_restaurant_Votes', 'median_restaurant_Rating',
       'median_restaurant_Reviews', 'median_restaurant_Votes', 'new_city',
       'restaurant_type'],
      dtype='object', length=118)


((7763, 118), (7763,), (3328, 118), (3328,))

# Random Forest

# Model-1 Validation

In [237]:
np.random.seed(100)
RF = RandomForestClassifier(n_estimators = 280)
RF.fit(train_local_X, train_local_Y)
RF_local_prediction = RF.predict(test_local_X)

print(accuracy_score(RF_local_prediction, test_local_Y))

0.8097956730769231


# Model-1 Final

In [71]:
np.random.seed(100)
RF = RandomForestClassifier(n_estimators=280)
RF.fit(train_prod[indep], train_prod[dep_column])
RF_prod_prediction_9 = RF.predict_proba(test_prod[indep])

# RF 20

In [72]:
# Splitting the data to local train and test
not_in_testset = ['Tex-Mex', 'Tamil', 'Sri Lankan', 'South American', 'Portuguese', 
                  'Nepalese', 'Konkan', 'Israeli', 'Indonesian', 'Greek', 'Gujarati', 
                  'Charcoal Chicken', 'Cantonese', 'Bubble Tea','Bohri']

indep = DF_prod.columns.difference([dep_column
                                    ,'Cusine_len'
                                    , 'median_restaurant_Rating'
                                    , 'median_restaurant_Reviews'
                                    , 'median_restaurant_Votes'
                                    , 'restaurant_type'
                                    , 'Location'
                                    ] 
                                   + not_in_testset
                                  )



#print(np.sort(indep).tolist())
print(indep)

np.random.seed(100)
train_local_X, test_local_X, train_local_Y, test_local_Y = train_test_split(train_prod[indep], 
                                                                            train_prod[dep_column],
                                                                            test_size = 0.3)

train_local_X.shape, train_local_Y.shape, test_local_X.shape, test_local_Y.shape

Index(['Afghan', 'African', 'American', 'Andhra', 'Arabian', 'Asian',
       'Assamese', 'Average_Cost', 'Awadhi', 'BBQ', 'Bakery', 'Bangladeshi',
       'Bar Food', 'Belgian', 'Bengali', 'Beverages', 'Bihari', 'Biryani',
       'Brazilian', 'Burger', 'Burmese', 'Cafe', 'Chettinad', 'Chinese',
       'Coffee', 'Continental', 'Cuisines', 'Desserts', 'European',
       'Fast Food', 'Finger Food', 'French', 'Frozen Yogurt', 'German', 'Goan',
       'Healthy Food', 'Hot dogs', 'Hyderabadi', 'Ice Cream', 'Indian',
       'Iranian', 'Italian', 'Japanese', 'Juices', 'Kashmiri', 'Kebab',
       'Kerala', 'Korean', 'Lebanese', 'Lucknowi', 'Maharashtrian',
       'Malaysian', 'Malwani', 'Mangalorean', 'Mediterranean', 'Mexican',
       'Middle Eastern', 'Minimum_Order', 'Mishti', 'Mithai', 'Modern Indian',
       'Momos', 'Mughlai', 'Naga', 'North Eastern', 'North Indian', 'Odia',
       'Paan', 'Parsi', 'Pizza', 'Poke', 'Rajasthani', 'Rating', 'Raw Meats',
       'Restaurant', 'Reviews', 'Roast

((7763, 99), (7763,), (3328, 99), (3328,))

# Model-2 Validation

In [240]:
np.random.seed(100)
RF = RandomForestClassifier(n_estimators = 220, n_jobs = -1)
RF.fit(train_local_X, train_local_Y)
RF_local_prediction = RF.predict(test_local_X)

print(accuracy_score(RF_local_prediction, test_local_Y))

0.8203125


# # Model-2 Final

In [73]:
np.random.seed(100)
RF = RandomForestClassifier(n_estimators=220)
RF.fit(train_prod[indep], train_prod[dep_column])
RF_prod_prediction_20 = RF.predict_proba(test_prod[indep])

# Ensemble

In [74]:
ensemble = np.argmax(np.mean([RF_prod_prediction_9
                              ,RF_prod_prediction_20
                             ], axis = 0), axis = 1)

Ensemble_prod_prediction = Dep_encoder.inverse_transform(ensemble)

sub = pd.DataFrame({'Delivery_Time' : Ensemble_prod_prediction})
sub.to_excel('sub_ENS_RF9_RF20_test.xlsx', index = False)